# Reading, cleaning and querying Pinterest Data from mounted S3 bucket using Sparks

In [ ]:
%run "/Users/amysw13@gmail.com/Mount S3 bucket to Databricks"

In [ ]:
# pyspark functions
from pyspark.sql.functions import *
from pyspark.sql.types import *


## Check mounted S3 bucket

In [ ]:
display(dbutils.fs.ls("/mnt/s3_pin_bucket_124714cdee67/topics/124714cdee67.geo/partition=0/"))

### Set spark databricks to not check for delta formats

In [ ]:
%sql
SET spark.databricks.delta.formatCheck.enabled=false

### Reading in mounted s3 bucket data

Each table read into three seperate dataframes (pin, geo and user). 

In [ ]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
pin_file_location = "/mnt/s3_pin_bucket_124714cdee67/topics/124714cdee67.pin/partition=0/124714cdee67.pin+0+*.json" 
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
pin_df = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(pin_file_location)
# Display Spark dataframe to check its content
display(pin_df)

In [ ]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
geo_file_location = "/mnt/s3_pin_bucket_124714cdee67/topics/124714cdee67.geo/partition=0/124714cdee67.geo+0+*.json" 
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
geo_df = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(geo_file_location)
# Display Spark dataframe to check its content
display(geo_df)

In [ ]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
user_file_location = "/mnt/s3_pin_bucket_124714cdee67/topics/124714cdee67.user/partition=0/124714cdee67.user+0+*.json" 
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
user_df = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(user_file_location)
# Display Spark dataframe to check its content
display(user_df)

## Cleaning Pinterest Post Data
To clean the df_pin DataFrame you should perform the following transformations:

1. Replace empty entries and entries with no relevant data in each column with Nones
2. Perform the necessary transformations on the follower_count to ensure every entry is a number. Make sure the data type of this column is an int.
3. Ensure that each column containing numeric data has a numeric data type
4. Clean the data in the save_location column to include only the save location path
5. Rename the index column to ind.
6. Reorder the DataFrame columns to have the following column order:
- ind
- unique_id
- title
- description
- follower_count
- poster_name
- tag_list
- is_image_or_video
- image_src
- save_location
- category

In [ ]:
# drop dulpicate rows
pin_df = pin_df.dropDuplicates()

In [ ]:
#check number of rows after dropping duplicates
row = pin_df.count()
print(f'Number of Rows are: {row}')
# 204 to 88 rows after dropping duplicate rows produced from streaming data multiple times during testing

In [ ]:
display(pin_df)

In [ ]:
# Replacing missing entries and irrelevant data with None
clean_pin_df = pin_df.replace({'User Info Error': None}, subset=['follower_count'])
clean_pin_df = clean_pin_df.replace({'No description available Story format': None}, subset=['description'])
clean_pin_df = clean_pin_df.replace({'Image src error.': None}, subset=['image_src'])
clean_pin_df = clean_pin_df.replace({'N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e': None}, subset=['tag_list'])
clean_pin_df = clean_pin_df.replace({'No Title Data Available': None}, subset=['title'])

In [ ]:
# Cast 'follower_count' to integer data type, but first convert any "k" and "M" to number
clean_pin_df = clean_pin_df.withColumn(
    "follower_count",
    when(
        col("follower_count").contains("k"),
        regexp_extract(col("follower_count"), "(\d+(.\d+)?)", 1).cast(DoubleType())
        * 1000
    )
    .when(
        col("follower_count").contains("M"),
        regexp_extract(col("follower_count"), "(\d+(.\d+)?)", 1).cast(DoubleType())
        * 1000000
    )
    .otherwise(regexp_extract(col("follower_count"), "(\d+(.\d+)?)", 1).cast("integer"))
    .cast("integer")
)

In [ ]:
display(clean_pin_df)

In [ ]:
# cast any numeric columns to a numeric data type 
clean_pin_df = clean_pin_df.withColumn("downloaded", clean_pin_df["downloaded"].cast("integer")) \
      .withColumn("index", clean_pin_df["index"].cast("integer"))

In [ ]:
# clean save location column to contain only relative path
clean_pin_df = clean_pin_df.withColumn("save_location", regexp_replace("save_location", "Local save in ", ""))

In [ ]:
clean_pin_df = clean_pin_df.withColumnRenamed("index", "ind")

In [ ]:
# reorder columns of cleaned pinterest data dataframe
clean_pin_df = clean_pin_df.select("ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list", "is_image_or_video", "image_src", "save_location", "category")

clean_pin_df.printSchema()

## Cleaning geolocation data

1. Create a new column coordinates that contains an array based on the latitude and longitude columns
2. Drop the latitude and longitude columns from the DataFrame
3. Convert the timestamp column from a string to a timestamp data type
4. Reorder the DataFrame columns to have the following column order:
 - ind
 - country
 - coordinates
 - timestamp

In [ ]:
# drop dulpicate rows
geo_df = geo_df.dropDuplicates()

In [ ]:
#check number of rows after dropping duplicates
row = geo_df.count()
print(f'Number of Rows are: {row}')
# 202 to 88 rows after dropping duplicate rows produced from streaming data multiple times during testing

In [ ]:
display(geo_df)

In [ ]:
clean_geo_df = geo_df \
    .withColumn("coordinates", array("latitude", "longitude")) \
    .drop("latitude", "longitude") \
    .withColumn("timestamp", col("timestamp").cast(TimestampType())) \
    .select("ind", "country", "coordinates", "timestamp")        

In [ ]:
display(clean_geo_df)

## Cleaning user data

1. Create a new column user_name that concatenates the information found in the first_name and last_name columns
2. Drop the first_name and last_name columns from the DataFrame
3. Convert the date_joined column from a string to a timestamp data type
4. Reorder the DataFrame columns to have the following column order:
 - ind
 - user_name
 - age
 - date_joined

In [ ]:
# drop dulpicate rows
user_df = user_df.dropDuplicates()

In [ ]:
#check number of rows after dropping duplicates
row = user_df.count()
print(f'Number of Rows are: {row}')
# 205 to 88 rows after dropping duplicate rows produced from streaming data multiple times during testing

In [ ]:
display(user_df)

In [ ]:
clean_user_df = user_df \
    .withColumn("user_name", concat("first_name",  lit(" "), "last_name")) \
    .drop("first_name", "last_name") \
    .withColumn("date_joined", col("date_joined").cast(TimestampType())) \
    .select("ind", "user_name", "age", "date_joined")

In [ ]:
display(clean_user_df)

## Querying Pinterest Data


Find the most popular Pinterest category people post to based on their country.


Your query should return a DataFrame that contains the following columns:

 - country
 - category
 - category_count, (a new column containing the desired query output)

In [ ]:
# Join all cleaned dataframes together by ind
joined_df = clean_pin_df.join(clean_geo_df, ["ind"]) \
            .join(clean_user_df, ["ind"])

In [ ]:
display(joined_df)

Find the most popular Pinterest category people post to based on their country.


Your query should return a DataFrame that contains the following columns:

 - country
 - category
 - category_count, (a new column containing the desired query output)

In [ ]:
popular_category = joined_df.groupBy("country", "category").agg(count("category").alias("category_count")).orderBy("category_count", ascending=False)
display(popular_category)

Find how many posts each category had between 2018 and 2022.

Your query should return a DataFrame that contains the following columns:

 - post_year, a new column that contains only the year from the timestamp column
 - category
 - category_count, a new column containing the desired query output

In [ ]:
category_num_post = joined_df \
    .withColumn("post_year", year('timestamp')) \
    .groupBy("post_year", "category") \
    .agg(count("category").alias("category_count")) \
    .orderBy("category_count", ascending=False)

display(category_num_post)

1. For each country find the user with the most followers.

Your query should return a DataFrame that contains the following columns:
 - country
 - poster_name
 - follower_count

2. Based on the above query, find the country with the user with most followers.

Your query should return a DataFrame that contains the following columns:

 - country
 - follower_count

This DataFrame should have only one entry.

In [ ]:
from pyspark.sql.window import Window

windowSpec = Window.partitionBy("country").orderBy(col("follower_count").desc())

user_country_followers = joined_df \
    .withColumn("max_follower_count", max("follower_count").over(windowSpec)) \
    .where(col("follower_count") == col("max_follower_count")) \
    .select("country", "user_name", "follower_count") \
    .dropDuplicates() 

display(user_country_followers)

In [ ]:
highest_follower_country = user_country_followers \
    .groupBy("country") \
    .agg(max("follower_count").alias("follower_count")) \
    .orderBy(desc("follower_count")) \
    .limit(1)

display(highest_follower_country)

What is the most popular category people post to based on the following age groups:

 - 18-24
 - 25-35
 - 36-50
 - +50

Your query should return a DataFrame that contains the following columns:

 - age_group, a new column based on the original age column
 - category
 - category_count, a new column containing the desired query output

In [ ]:
# Define a custom sorting order for cities
age_order = ["18 - 24", "25 - 35", "36 - 50", "+50"]

# Create a custom sorting column for "age_group"
custom_sort_col = when(col("age_group") == age_order[0], 0) \
    .when(col("age_group") == age_order[1], 1) \
    .when(col("age_group") == age_order[2], 2) \
    .otherwise(3)
    
popular_category_age = joined_df \
    .withColumn("age_group", when((joined_df.age >= 18) & (joined_df.age <= 24), "18 - 24")
                .when((joined_df.age >= 25) & (joined_df.age <= 35), "25 - 35")
                .when((joined_df.age >= 36) & (joined_df.age <= 50), "36 - 50")
                .otherwise("+50")) \
    .groupBy("age_group", "category") \
    .agg(count("category").alias("category_count")) \
    .orderBy(custom_sort_col, "category_count", ascending=False)

display(popular_category_age)
    

What is the median follower count for users in the following age groups:

 - 18-24
 - 25-35
 - 36-50
 - +50

Your query should return a DataFrame that contains the following columns:

 - age_group, a new column based on the original age column
 - median_follower_count, a new column containing the desired query output

In [ ]:
median_followers_count = joined_df \
    .withColumn("age_group", when((joined_df.age >= 18) & (joined_df.age <= 24), '18 - 24')
                .when((joined_df.age >= 25) & (joined_df.age <= 35), '25 - 35')
                .when((joined_df.age >= 36) & (joined_df.age <= 50), '36 - 50')
                .otherwise('+50')) \
    .groupBy("age_group") \
    .agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
    .orderBy("median_follower_count", ascending=False)

display(median_followers_count )

Find how many users have joined between 2015 and 2020.


Your query should return a DataFrame that contains the following columns:

 - post_year, a new column that contains only the year from the timestamp column
 - number_users_joined, a new column containing the desired query output

In [ ]:
users_year_joined = joined_df \
    .withColumn("join_year", year('date_joined')) \
    .groupBy("join_year") \
    .agg(count("user_name").alias("numbers_users_joined")) \
    .orderBy("numbers_users_joined", ascending=False)

display(users_year_joined)

Find the median follower count of users have joined between 2015 and 2020.


Your query should return a DataFrame that contains the following columns:

 - post_year, a new column that contains only the year from the timestamp column
 - median_follower_count, a new column containing the desired query output

In [ ]:
med_follower_year = joined_df \
    .withColumn("join_year", year('date_joined')) \
    .groupBy("join_year") \
    .agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
    .orderBy("median_follower_count", ascending=False)

display(med_follower_year)

Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of.


Your query should return a DataFrame that contains the following columns:

 - age_group, a new column based on the original age column
 - post_year, a new column that contains only the year from the timestamp column
 - median_follower_count, a new column containing the desired query output

In [ ]:
med_follower_year_agegroup = joined_df \
    .withColumn("age_group", when((joined_df.age >= 18) & (joined_df.age <= 24), '18 - 24')
            .when((joined_df.age >= 25) & (joined_df.age <= 35), '25 - 35')
            .when((joined_df.age >= 36) & (joined_df.age <= 50), '36 - 50')
            .otherwise('+50')) \
    .withColumn("join_year", year('date_joined')) \
    .groupBy("age_group","join_year") \
    .agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
    .orderBy(custom_sort_col, "median_follower_count")

display(med_follower_year_agegroup)

### Unmount S3 bucket from Databricks

In [ ]:
# Mount name for the bucket
MOUNT_NAME = "/mnt/s3_pin_bucket_124714cdee67"
# to unmount
dbutils.fs.unmount(MOUNT_NAME)